In [19]:
%%capture
!pip install ultralytics

In [20]:
# === IMPORTACIONES ===
from ultralytics import YOLO
import os
import shutil
from sklearn.model_selection import KFold
import pandas as pd
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Entrenamienot de Yolo (10, 11, 12)

In [3]:


resultados_folds = {}
confidence_thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
num_folds = 10
epochs=200

imgsz=640
ruta_base_resultados = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo10'

for fold in range(num_folds):
    print(f"🔁 Fold {fold+1}")

    resultados_folds[fold+1] = {}

    # a. Ruta del archivo YAML para este fold
    # /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11
    data_yaml_fold_path = f"/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold{fold}/data_fold{fold}.yaml"

    # b. Ruta para guardar resultados
    project_path_train = os.path.join(ruta_base_resultados, f"results_train_fold{fold+1}")
    name_train = f"fold{fold+1}"

    # c. Entrenar desde cero con archivo YAML
    model = YOLO("yolov10s.yaml")  # O el YAML que estés usando
    model.train(
        optimizer='AdamW',
        data=data_yaml_fold_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        patience=20,
        project=project_path_train,
        name=name_train,
        plots=False
    )

    print(f"✅ Entrenamiento del Fold {fold+1} terminado.")

    # d. Evaluar con múltiples thresholds
    resultados_thresholds = {}
    best_weights_path = os.path.join(project_path_train, name_train, "weights", "best.pt")

    if os.path.exists(best_weights_path):
        for conf_threshold in confidence_thresholds:
            print(f"➡️ Evaluando Fold {fold+1} con threshold {conf_threshold}")
            val_results = model.val(
                data=data_yaml_fold_path,
                imgsz=imgsz,
                conf=conf_threshold,
                verbose=False,
                split='val',
                device=0,
                model=best_weights_path
            )
            metrics = val_results.results_dict
            resultados_thresholds[conf_threshold] = {
                'mAP50': metrics.get('metrics/mAP50(B)', 0.0),
                'mAP': metrics.get('metrics/mAP50-95(B)', 0.0),
                'precision': metrics.get('metrics/precision(B)', 0.0),
                'recall': metrics.get('metrics/recall(B)', 0.0)
            }
            print(f"   ✅ Threshold {conf_threshold} | mAP@0.5: {resultados_thresholds[conf_threshold]['mAP50']:.4f}")
    else:
        print(f"⚠️ No se encontraron pesos para el Fold {fold+1}")
        for conf_threshold in confidence_thresholds:
            resultados_thresholds[conf_threshold] = None

    resultados_folds[fold+1] = resultados_thresholds

# --- Guardar resultados en CSV ---
all_fold_data = []
for fold, threshold_data in resultados_folds.items():
    for threshold, metrics in threshold_data.items():
        if metrics:
            all_fold_data.append({
                'Fold': fold,
                'Threshold': threshold,
                'mAP50': metrics.get('mAP50', 0.0),
                'mAP': metrics.get('mAP', 0.0),
                'Precision': metrics.get('precision', 0.0),
                'Recall': metrics.get('recall', 0.0)
            })

df_resultados_folds = pd.DataFrame(all_fold_data)
csv_path = os.path.join(ruta_base_resultados, "resultados_folds_yolov10.csv")
df_resultados_folds.to_csv(csv_path, index=False)
print(f"📁 Resultados guardados en: {csv_path}")


🔁 Fold 1
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/data_fold0.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold1, nbs=64, nms=False, opset=None, optimize=False

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1     36096  ultralytics.nn.modules.block.SCDown          [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1    137728  ultralytics

YOLOv10s summary: 234 layers, 8,067,126 parameters, 8,067,110 gradients, 24.8 GFLOPs

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 63.5MB/s]


...

val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold9/labels/val.cache... 202 images, 0 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:00<00:00, 13.42it/s]

                   all        202        215      0.984      0.293      0.637      0.481
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.1ms postprocess per image


   ✅ Threshold 0.8 | mAP@0.5: 0.6369
➡️ Evaluando Fold 10 con threshold 0.9
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 241.0±113.1 MB/s, size: 411.3 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold9/labels/val.cache... 202 images, 0 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:00<00:00, 13.71it/s]

                   all        202        215      0.947     0.0837      0.515      0.411
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.1ms postprocess per image
   ✅ Threshold 0.9 | mAP@0.5: 0.5152
📁 Resultados guardados en: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo10/resultados_folds_yolov10.csv


In [6]:


resultados_folds = {}
confidence_thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
num_folds = 10
epochs=200

imgsz=640
ruta_base_resultados = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11'

for fold in range(num_folds):
    print(f"🔁 Fold {fold+1}")

    resultados_folds[fold+1] = {}

    # a. Ruta del archivo YAML para este fold
    # /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11
    data_yaml_fold_path = f"/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold{fold}/data_fold{fold}.yaml"

    # b. Ruta para guardar resultados
    project_path_train = os.path.join(ruta_base_resultados, f"results_train_fold{fold+1}")
    name_train = f"fold{fold+1}"

    # c. Entrenar desde cero con archivo YAML
    model = YOLO("yolo11s.yaml")
    model.train(
        optimizer='AdamW',
        data=data_yaml_fold_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        patience=20,
        project=project_path_train,
        name=name_train,
        plots=True
    )

    print(f"✅ Entrenamiento del Fold {fold+1} terminado.")

    # d. Evaluar con múltiples thresholds
    resultados_thresholds = {}
    best_weights_path = os.path.join(project_path_train, name_train, "weights", "best.pt")

    if os.path.exists(best_weights_path):
        for conf_threshold in confidence_thresholds:
            print(f"➡️ Evaluando Fold {fold+1} con threshold {conf_threshold}")
            val_results = model.val(
                data=data_yaml_fold_path,
                imgsz=imgsz,
                conf=conf_threshold,
                verbose=False,
                split='val',
                device=0,
                model=best_weights_path
            )
            metrics = val_results.results_dict
            resultados_thresholds[conf_threshold] = {
                'mAP50': metrics.get('metrics/mAP50(B)', 0.0),
                'mAP': metrics.get('metrics/mAP50-95(B)', 0.0),
                'precision': metrics.get('metrics/precision(B)', 0.0),
                'recall': metrics.get('metrics/recall(B)', 0.0)
            }
            print(f"   ✅ Threshold {conf_threshold} | mAP@0.5: {resultados_thresholds[conf_threshold]['mAP50']:.4f}")
    else:
        print(f"⚠️ No se encontraron pesos para el Fold {fold+1}")
        for conf_threshold in confidence_thresholds:
            resultados_thresholds[conf_threshold] = None

    resultados_folds[fold+1] = resultados_thresholds

# --- Guardar resultados en CSV ---
all_fold_data = []
for fold, threshold_data in resultados_folds.items():
    for threshold, metrics in threshold_data.items():
        if metrics:
            all_fold_data.append({
                'Fold': fold,
                'Threshold': threshold,
                'mAP50': metrics.get('mAP50', 0.0),
                'mAP': metrics.get('mAP', 0.0),
                'Precision': metrics.get('precision', 0.0),
                'Recall': metrics.get('recall', 0.0)
            })

df_resultados_folds = pd.DataFrame(all_fold_data)
csv_path = os.path.join(ruta_base_resultados, "resultados_folds_yolov11.csv")
df_resultados_folds.to_csv(csv_path, index=False)
print(f"📁 Resultados guardados en: {csv_path}")


🔁 Fold 1
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/data_fold0.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold1, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/labels/train.cache... 1821 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1822/1822 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/images/train/29370796_45836254.jpeg: ignoring corrupt image/label: image file is truncated (3 bytes not processed)
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 7.7±14.0 ms, read: 39.3±43.8 MB/s, size: 267.1 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/labels/val.cache... 203 images, 0 backgrounds, 0 corrupt: 100%|██████████| 203/203 [00:00<?, ?it/s]


...

                   all        202        215          1      0.307      0.653      0.468
Speed: 1.0ms preprocess, 1.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11/results_train_fold10/fold104
   ✅ Threshold 0.7 | mAP@0.5: 0.6535
➡️ Evaluando Fold 10 con threshold 0.8
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 143.7±43.9 MB/s, size: 244.8 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold9/labels/val.cache... 202 images, 0 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  8.88it/s]


                   all        202        215          0          0          0          0
Speed: 1.7ms preprocess, 1.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11/results_train_fold10/fold105
   ✅ Threshold 0.8 | mAP@0.5: 0.0000
➡️ Evaluando Fold 10 con threshold 0.9
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 189.4±90.8 MB/s, size: 411.3 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold9/labels/val.cache... 202 images, 0 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  8.70it/s]


                   all        202        215          0          0          0          0
Speed: 1.8ms preprocess, 1.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11/results_train_fold10/fold106
   ✅ Threshold 0.9 | mAP@0.5: 0.0000
📁 Resultados guardados en: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11/resultados_folds_yolov11.csv


In [7]:


resultados_folds = {}
confidence_thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
num_folds = 10
epochs=200

imgsz=640
ruta_base_resultados = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo12'

for fold in range(num_folds):
    print(f"🔁 Fold {fold+1}")

    resultados_folds[fold+1] = {}

    # a. Ruta del archivo YAML para este fold
    # /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11
    data_yaml_fold_path = f"/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold{fold}/data_fold{fold}.yaml"

    # b. Ruta para guardar resultados
    project_path_train = os.path.join(ruta_base_resultados, f"results_train_fold{fold+1}")
    name_train = f"fold{fold+1}"

    # c. Entrenar desde cero con archivo YAML
    model = YOLO("yolo12s.yaml")
    model.train(
        optimizer='AdamW',
        data=data_yaml_fold_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=16,
        patience=20,
        project=project_path_train,
        name=name_train,
        plots=True
    )

    print(f"✅ Entrenamiento del Fold {fold+1} terminado.")

    # d. Evaluar con múltiples thresholds
    resultados_thresholds = {}
    best_weights_path = os.path.join(project_path_train, name_train, "weights", "best.pt")

    if os.path.exists(best_weights_path):
        for conf_threshold in confidence_thresholds:
            print(f"➡️ Evaluando Fold {fold+1} con threshold {conf_threshold}")
            val_results = model.val(
                data=data_yaml_fold_path,
                imgsz=imgsz,
                conf=conf_threshold,
                verbose=False,
                split='val',
                device=0,
                model=best_weights_path
            )
            metrics = val_results.results_dict
            resultados_thresholds[conf_threshold] = {
                'mAP50': metrics.get('metrics/mAP50(B)', 0.0),
                'mAP': metrics.get('metrics/mAP50-95(B)', 0.0),
                'precision': metrics.get('metrics/precision(B)', 0.0),
                'recall': metrics.get('metrics/recall(B)', 0.0)
            }
            print(f"   ✅ Threshold {conf_threshold} | mAP@0.5: {resultados_thresholds[conf_threshold]['mAP50']:.4f}")
    else:
        print(f"⚠️ No se encontraron pesos para el Fold {fold+1}")
        for conf_threshold in confidence_thresholds:
            resultados_thresholds[conf_threshold] = None

    resultados_folds[fold+1] = resultados_thresholds

# --- Guardar resultados en CSV ---
all_fold_data = []
for fold, threshold_data in resultados_folds.items():
    for threshold, metrics in threshold_data.items():
        if metrics:
            all_fold_data.append({
                'Fold': fold,
                'Threshold': threshold,
                'mAP50': metrics.get('mAP50', 0.0),
                'mAP': metrics.get('mAP', 0.0),
                'Precision': metrics.get('precision', 0.0),
                'Recall': metrics.get('recall', 0.0)
            })

df_resultados_folds = pd.DataFrame(all_fold_data)
csv_path = os.path.join(ruta_base_resultados, "resultados_folds_yolov12.csv")
df_resultados_folds.to_csv(csv_path, index=False)
print(f"📁 Resultados guardados en: {csv_path}")


🔁 Fold 1
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/data_fold0.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold1, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/labels/train.cache... 1821 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1822/1822 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/images/train/29370796_45836254.jpeg: ignoring corrupt image/label: image file is truncated (3 bytes not processed)
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 2.1±2.9 ms, read: 43.6±46.0 MB/s, size: 267.1 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold0/labels/val.cache... 203 images, 0 backgrounds, 0 corrupt: 100%|██████████| 203/203 [00:00<?, ?it/s]


...

                   all        202        215          1       0.13      0.565      0.402
Speed: 1.2ms preprocess, 2.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo12/results_train_fold10/fold104
   ✅ Threshold 0.7 | mAP@0.5: 0.5651
➡️ Evaluando Fold 10 con threshold 0.8
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 121.0±46.4 MB/s, size: 244.8 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold9/labels/val.cache... 202 images, 0 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  7.87it/s]


                   all        202        215          0          0          0          0
Speed: 1.4ms preprocess, 2.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo12/results_train_fold10/fold105
   ✅ Threshold 0.8 | mAP@0.5: 0.0000
➡️ Evaluando Fold 10 con threshold 0.9
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 180.6±91.3 MB/s, size: 411.3 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/outputs/kfold_yolo_11/fold9/labels/val.cache... 202 images, 0 backgrounds, 0 corrupt: 100%|██████████| 202/202 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  8.12it/s]


                   all        202        215          0          0          0          0
Speed: 1.8ms preprocess, 2.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo12/results_train_fold10/fold106
   ✅ Threshold 0.9 | mAP@0.5: 0.0000
📁 Resultados guardados en: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo12/resultados_folds_yolov12.csv


## Tabla

In [3]:
import os
import pandas as pd
import numpy as np

# Ruta donde se guardaron los CSVs de validación por modelo
ruta_base_resultados = "/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch"

# Nombres reales de los modelos y sus CSVs correspondientes
model_names = {
    'yolov10s': '/yolo10/resultados_folds_yolov10.csv',
    'yolo11s': '/yolo11/resultados_folds_yolov11.csv',
    'yolo12s': '/yolo12/resultados_folds_yolov12.csv'
}

# Umbrales usados
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]

# Diccionario para guardar resultados por modelo
resultados_por_modelo = {}

# Procesar cada CSV
for model_name, csv_name in model_names.items():
    csv_path = ruta_base_resultados +  csv_name
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        resultados_por_threshold = {}
        for threshold in thresholds:
            df_threshold = df[df['Threshold'] == threshold]
            if not df_threshold.empty:
                promedio_map50 = df_threshold['mAP50'].mean()
                desviacion_map50 = df_threshold['mAP50'].std()
                promedio_map = df_threshold['mAP'].mean()
                desviacion_map = df_threshold['mAP'].std()
                promedio_precision = df_threshold['Precision'].mean()
                desviacion_precision = df_threshold['Precision'].std()
                promedio_recall = df_threshold['Recall'].mean()
                desviacion_recall = df_threshold['Recall'].std()

                resultados_por_threshold[threshold] = {
                    'mAP50': (promedio_map50, desviacion_map50),
                    'mAP': (promedio_map, desviacion_map),
                    'Precision': (promedio_precision, desviacion_precision),
                    'Recall': (promedio_recall, desviacion_recall)
                }
            else:
                resultados_por_threshold[threshold] = {
                    'mAP50': (None, None),
                    'mAP': (None, None),
                    'Precision': (None, None),
                    'Recall': (None, None)
                }
        resultados_por_modelo[model_name] = resultados_por_threshold
    else:
        print(f"⚠️ No se encontró el archivo CSV para {model_name} en {ruta_base_resultados}")

# Mostrar resultados y guardar en CSV
for model, resultados_threshold in resultados_por_modelo.items():
    print(f"\n--- Resultados Promedio y Desviación Estándar para {model} ---")
    data = []
    index = []
    for threshold, metrics in resultados_threshold.items():
        index.append(f"Threshold {threshold}")
        row = []
        for metric_name, (mean, std) in metrics.items():
            if mean is not None:
                row.append(f"{mean:.3f} ({std:.3f})")
            else:
                row.append("N/A")
        data.append(row)

    df_resultados = pd.DataFrame(data, index=index, columns=['mAP50', 'mAP', 'Precision', 'Recall'])
    print(df_resultados.to_string(index=True, justify='center'))

    # Guardar en CSV
    output_csv_path = os.path.join(ruta_base_resultados, f"resumen_{model}.csv")
    df_resultados.to_csv(output_csv_path)
    print(f"✅ Guardado: {output_csv_path}")



--- Resultados Promedio y Desviación Estándar para yolov10s ---
                  mAP50           mAP         Precision        Recall    
Threshold 0.5  0.881 (0.048)  0.573 (0.042)  0.947 (0.021)  0.793 (0.088)
Threshold 0.6  0.841 (0.057)  0.554 (0.045)  0.970 (0.016)  0.704 (0.109)
Threshold 0.7  0.782 (0.071)  0.525 (0.050)  0.981 (0.009)  0.580 (0.139)
Threshold 0.8  0.683 (0.082)  0.478 (0.051)  0.986 (0.014)  0.381 (0.158)
Threshold 0.9  0.461 (0.162)  0.353 (0.127)  0.890 (0.313)  0.032 (0.028)
✅ Guardado: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/resumen_yolov10s.csv

--- Resultados Promedio y Desviación Estándar para yolo11s ---
                  mAP50           mAP         Precision        Recall    
Threshold 0.5  0.918 (0.046)  0.585 (0.041)  0.956 (0.021)  0.855 (0.084)
Threshold 0.6  0.876 (0.064)  0.567 (0.046)  0.979 (0.013)  0.762 (0.124)
Threshold 0.7  0.714 (0.097)  0.482 (0.060)  0.996 (0.007)  0.432 (0.194)
Threshold 0.8  0.41

## Prueba estadistica

In [5]:
import pandas as pd
import os
from scipy.stats import wilcoxon, mannwhitneyu

# Ruta base
ruta_base_resultados = "/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch"

# Nombres reales de los modelos y sus CSVs correspondientes
modelos = {
    'yolov10s': '/yolo10/resultados_folds_yolov10.csv',
    'yolo11s': '/yolo11/resultados_folds_yolov11.csv',
    'yolo12s': '/yolo12/resultados_folds_yolov12.csv'
}

thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]

# Paso 1: Cargar datos
df_modelos = {}
for modelo, archivo in modelos.items():
    path_csv = ruta_base_resultados + archivo
    if os.path.exists(path_csv):
        df_modelos[modelo] = pd.read_csv(path_csv)

# Paso 2: Encontrar mejor mAP50 promedio
mejor_modelo = None
mejor_threshold = None
mejor_map50 = -1

for modelo, df in df_modelos.items():
    for threshold in thresholds:
        mean_map = df[df['Threshold'] == threshold]['mAP50'].mean()
        if mean_map > mejor_map50:
            mejor_map50 = mean_map
            mejor_modelo = modelo
            mejor_threshold = threshold

print("🔍 Mejor combinación encontrada:")
print(f"Modelo: {mejor_modelo}")
print(f"Threshold: {mejor_threshold}")
print(f"Promedio mAP50: {mejor_map50:.4f}")

# Obtener valores del modelo pivote
valores_pivote = df_modelos[mejor_modelo][df_modelos[mejor_modelo]['Threshold'] == mejor_threshold]['mAP50'].values

# Paso 3: Comparaciones solo contra el pivote
comparaciones = []

for modelo, df in df_modelos.items():
    for threshold in thresholds:
        # Saltar la comparación contra sí mismo
        if modelo == mejor_modelo and threshold == mejor_threshold:
            continue

        valores_comparar = df[df['Threshold'] == threshold]['mAP50'].values

        if len(valores_comparar) > 0:
            try:
                # U-test
                u_stat, p_u = mannwhitneyu(valores_pivote, valores_comparar, alternative='two-sided')

                # Wilcoxon solo si las longitudes coinciden
                if len(valores_pivote) == len(valores_comparar):
                    w_stat, p_w = wilcoxon(valores_pivote, valores_comparar)
                else:
                    p_w = None

                comparaciones.append({
                    "Pivote_Modelo": mejor_modelo,
                    "Pivote_Threshold": mejor_threshold,
                    "Comparado_Modelo": modelo,
                    "Comparado_Threshold": threshold,
                    "p_value_Utest": p_u,
                    "p_value_Wilcoxon": p_w
                })

            except Exception as e:
                print(f"⚠️ Error comparando pivote vs {modelo}@{threshold}: {e}")

# Paso 4: Guardar resultados
df_comparaciones = pd.DataFrame(comparaciones)
df_comparaciones.sort_values(by='p_value_Utest', inplace=True)

ruta_guardado = os.path.join(ruta_base_resultados, "comparacion_pvalues_vs_pivote.csv")
df_comparaciones.to_csv(ruta_guardado, index=False)

print(f"\n✅ Comparaciones contra el pivote guardadas en: {ruta_guardado}")


🔍 Mejor combinación encontrada:
Modelo: yolo11s
Threshold: 0.5
Promedio mAP50: 0.9184

✅ Comparaciones contra el pivote guardadas en: /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/comparacion_pvalues_vs_pivote.csv


In [17]:
import os
import pandas as pd

# Ruta base donde están las carpetas resultados_fold1, resultados_fold2, etc.
base_path = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11'

# Diccionario para guardar los resultados
mejores_resultados = {}

# Iterar sobre carpetas como resultados_fold1, resultados_fold2, etc.
for i in range(10):
    fold_path = base_path + f'/results_train_fold{i+1}/fold{i+1}'
    csv_path = fold_path + '/results.csv'
    df = pd.read_csv(csv_path)
    ultimo = df.tail(1).iloc[0]
    key_name = 'fold' + str(i+1)
    mejores_resultados[key_name] = {
                        'mAP50': ultimo.get('metrics/mAP50(B)', 0.0),
                        'mAP50-95': ultimo.get('metrics/mAP50-95(B)', 0.0),
                        'precision': ultimo.get('metrics/precision(B)', 0.0),
                        'recall': ultimo.get('metrics/recall(B)', 0.0),
                    }


# Convertir a DataFrame para ordenarlo
df_resultados = pd.DataFrame.from_dict(mejores_resultados, orient='index')
df_ordenado = df_resultados.sort_values(by='mAP50', ascending=False)

# Mostrar ordenado
print("🏆 Mejores folds ordenados por mAP50:")
print(df_ordenado)

🏆 Mejores folds ordenados por mAP50:
          mAP50  mAP50-95  precision   recall
fold6   0.98365   0.59997    0.96108  0.93301
fold1   0.97531   0.58475    0.95428  0.92523
fold10  0.97118   0.59844    0.95917  0.91628
fold7   0.96828   0.55576    0.96632  0.93925
fold8   0.95919   0.57189    0.95506  0.90826
fold3   0.95364   0.55327    0.96578  0.91055
fold2   0.95135   0.54851    0.93586  0.90793
fold4   0.94097   0.53438    0.93210  0.87783
fold5   0.92246   0.51667    0.89989  0.87324
fold9   0.78721   0.42381    0.85667  0.71152


## Prueba

In [21]:
path_best_model = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11/results_train_fold6/fold6/weights/best.pt'
data_yaml = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/test.yaml'

model = YOLO(path_best_model)

# Validar el modelo con threshold de confianza 0.6
results = model.val(data=data_yaml, conf=0.6, iou=0.5, split='test')

# Acceder directamente a las métricas desde results


metrics_dict = {
    'mAP50': results.box.map50,
    'mAP50-95': results.box.map,
    'Recall': results.box.p,
    'Precision': results.box.r
}

# Mostrar el vector de métricas
print("Vector de métricas del Fold 5 con threshold 0.6:")
print(metrics_dict)

Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 13.3MB/s]


val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 1.9±1.4 MB/s, size: 500.2 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/data/merged/labels.cache... 870 images, 0 backgrounds, 0 corrupt: 100%|██████████| 870/870 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:30<00:00,  1.80it/s]


                   all        870       1074      0.975      0.757      0.869      0.551
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val
Vector de métricas del Fold 5 con threshold 0.6:
{'mAP50': np.float64(0.8685182507436415), 'mAP50-95': np.float64(0.5507643740769672), 'Recall': array([    0.97482]), 'Precision': array([    0.75698])}


In [22]:
path_best_model = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/resultados-scratch/yolo11/results_train_fold6/fold6/weights/best.pt'
data_yaml = '/content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/test.yaml'

model = YOLO(path_best_model)

# Validar el modelo con threshold de confianza 0.5
results = model.val(data=data_yaml, conf=0.5, iou=0.5, split='test')

# Acceder directamente a las métricas desde results


metrics_dict = {
    'mAP50': results.box.map50,
    'mAP50-95': results.box.map,
    'Recall': results.box.p,
    'Precision': results.box.r
}

# Mostrar el vector de métricas
print("Vector de métricas del Fold 5 con threshold 0.5:")
print(metrics_dict)

Ultralytics 8.3.129 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 124.8±78.6 MB/s, size: 329.2 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/personal-tests/tesis/data/merged/labels.cache... 870 images, 0 backgrounds, 0 corrupt: 100%|██████████| 870/870 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:05<00:00, 10.30it/s]


                   all        870       1074      0.967      0.791      0.884      0.556
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val2
Vector de métricas del Fold 5 con threshold 0.5:
{'mAP50': np.float64(0.8844081571717471), 'mAP50-95': np.float64(0.5560541519201376), 'Recall': array([    0.96701]), 'Precision': array([    0.79143])}
